Code to warm up kernel

In [2]:
#Imports
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from pytrends.request import TrendReq
#Declare constants
ETH_TICKER = "ETH-USD"
SPX_TICKER = "^GSPC" 
BTC_TICKER = "BTC-USD"

Below is the code to get data for:

Ether Vs S&P 500 data!

&

BTC Vs S&P 500 data!

In [3]:

START_DATE = "2022-01-01"

def clean_index(df):
    """Safely normalizes index and removes timezone if it exists."""
    df.index = df.index.normalize()
    if df.index.tz is not None:
        df.index = df.index.tz_localize(None)
    return df

# --- FETCH ETH DATA ---
# ETH trades 24/7, so this will be your primary (left) dataset
eth_data = yf.Ticker(ETH_TICKER).history(start=START_DATE, period="3y")
df_eth = eth_data[['Open', 'High', 'Low', 'Close','Volume']].copy()
df_eth = clean_index(df_eth)
    

df_combined_daily_eth = df_eth


    
# Display final data types and non-null counts
print("\n--- Final DataFrame Info ---")
df_combined_daily_eth.info()


--- Final DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1096 entries, 2022-01-01 to 2024-12-31
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    1096 non-null   float64
 1   High    1096 non-null   float64
 2   Low     1096 non-null   float64
 3   Close   1096 non-null   float64
 4   Volume  1096 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 51.4 KB


In [4]:
START_DATE = "2022-01-01"

def clean_index(df):
    """Safely normalizes index and removes timezone if it exists."""
    df.index = df.index.normalize()
    if df.index.tz is not None:
        df.index = df.index.tz_localize(None)
    return df

try:
    # --- FETCH BTC DATA ---
    # BTC trades 24/7, so this will be your primary (left) dataset
    btc_data = yf.Ticker(BTC_TICKER).history(start=START_DATE, period="3y")
    df_btc = btc_data[['Open', 'High', 'Low', 'Close','Volume']].copy()
    df_btc = clean_index(df_btc)
    

    df_combined_daily_btc = df_btc
        
except Exception as e:
    print(f"An error occurred while fetching data: {e}")
    exit()

# Display final data types and non-null counts
print("\n--- Final DataFrame Info ---")
df_combined_daily_btc.info()


--- Final DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1096 entries, 2022-01-01 to 2024-12-31
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    1096 non-null   float64
 1   High    1096 non-null   float64
 2   Low     1096 non-null   float64
 3   Close   1096 non-null   float64
 4   Volume  1096 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 51.4 KB


BTC HASHRATE + Fear & Greed index ~

All using dataset

In [5]:
FILE_PATH = "C:\\Users\\Kevin Ivander\\Documents\\GitHub\\DSC101\\csv_files\\hashrateBTC.csv"
START_DATE = '2022-01-01 00:00:00'
END_DATE = '2024-12-31 00:00:00'

df_hashrate_data = pd.read_csv(
    FILE_PATH,
    sep=';',
    names = ['DateTime', 'Hashrate'],
    index_col='DateTime',
    parse_dates=['DateTime'],
    skiprows=1, 
    header=None
    )
df_hashrate = df_hashrate_data.loc[START_DATE:END_DATE].copy()


df_hashrate.index = pd.to_datetime(df_hashrate.index)

df_hashrate['Hashrate'] = pd.to_numeric(df_hashrate['Hashrate'].astype(float))
daily_avg_hash = df_hashrate.resample('D').mean()
print("successfully extracted hashrate data")

df_combined_daily_btc.head()


successfully extracted hashrate data


,Open,High,Low,Close,Volume
Date,,,,,
2022-01-01,46311.746094,47827.312500,46288.484375,47686.812500,24582667004
2022-01-02,47680.925781,47881.406250,46856.937500,47345.218750,27951569547
2022-01-03,47343.542969,47510.726562,45835.964844,46458.117188,33071628362
2022-01-04,46458.851562,47406.546875,45752.464844,45897.574219,42494677905
2022-01-05,45899.359375,46929.046875,42798.222656,43569.003906,36851084859


In [6]:
import pandas as pd
FILE_PATH_FG = r'C:\Users\Kevin Ivander\Documents\GitHub\DSC101\csv_files\btc_with_fgi_4h.csv'
START_DATE = '2023-01-01 00:00:00'
END_DATE = '2024-12-31 00:00:00'
df_fng_data = pd.read_csv(
    FILE_PATH_FG,
    sep=',',
    index_col='timestamp',
    )
df_fng_data.index = pd.to_datetime(df_fng_data.index)                                   
df_fng_data.head()


,open,close,high,low,Fear & Greed Index,Fear & Greed Classification
timestamp,,,,,,
2025-10-02 04:00:00,118931.0,118774.0,119400.0,118439.0,64.0,Greed
2025-10-02 00:00:00,117403.0,118503.0,118503.0,117282.0,64.0,Greed
2025-10-01 20:00:00,117484.0,117481.0,118007.0,116853.0,49.0,Neutral
2025-10-01 16:00:00,116801.0,117392.0,117573.0,116435.0,49.0,Neutral
2025-10-01 12:00:00,114544.0,116644.0,116652.0,114544.0,49.0,Neutral


In [7]:
df_fng_data.drop(['open','close','high','low','Fear & Greed Classification'], axis=1, inplace=True)
df_fng_data.fillna(method='ffill', inplace=True)

C:\Users\Kevin Ivander\AppData\Local\Temp\ipykernel_14608\786426176.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_fng_data.fillna(method='ffill', inplace=True)


Google Trends data!

In [8]:
import pandas as pd
df_gtrends_daily = pd.read_csv(r'C:\Users\Kevin Ivander\Documents\GitHub\DSC101\csv_files\google_trends_daily_20251211.csv')
df_gtrends_daily['date'] = pd.to_datetime(df_gtrends_daily['date'])
df_gtrends_daily.set_index('date', inplace=True)
df_gtrends_daily.head()

,GTrends_BTC,GTrends_ETH
date,,
2022-01-01,45,12
2022-01-02,61,15
2022-01-03,61,15
2022-01-04,61,15
2022-01-05,61,15


In [9]:
df_gtrends_eth = df_gtrends_daily['GTrends_ETH'].copy()
df_gtrends_eth.head()

date
2022-01-01    12
2022-01-02    15
2022-01-03    15
2022-01-04    15
2022-01-05    15
Name: GTrends_ETH, dtype: int64

In [10]:
df_gtrends_btc = df_gtrends_daily['GTrends_BTC'].copy()
df_gtrends_btc.info()

<class 'pandas.core.series.Series'>
DatetimeIndex: 1094 entries, 2022-01-01 to 2024-12-29
Series name: GTrends_BTC
Non-Null Count  Dtype
--------------  -----
1094 non-null   int64
dtypes: int64(1)
memory usage: 17.1 KB


Sychronizing and Scaling

In [11]:
#BTC
df_combined_daily_btc = df_combined_daily_btc.join(df_fng_data)
df_combined_daily_btc = df_combined_daily_btc.join(daily_avg_hash)
df_combined_daily_btc = df_combined_daily_btc.join(df_gtrends_btc)
#ETH
df_combined_daily_eth = df_combined_daily_eth.join(df_fng_data)
df_combined_daily_eth = df_combined_daily_eth.join(df_gtrends_eth)

In [12]:

# Convert raw HashRate to Exahashes (EH/s) by dividing by 10^18 and volume into bilions
df_combined_daily_btc['Volume'] = df_combined_daily_btc['Volume'] / 1e10
df_combined_daily_eth['Volume'] = df_combined_daily_eth['Volume'] / 1e10
df_combined_daily_btc['Hashrate'] = df_combined_daily_btc['Hashrate'] / 1e18

# Rename the column so you don't forget the unit later
df_combined_daily_btc.rename(columns={'Hashrate': 'Hashrate_EH'}, inplace=True)
df_combined_daily_btc.rename(columns={'Volume': 'Volume (Bilions)'}, inplace=True)
df_combined_daily_eth.rename(columns={'Volume': 'Volume (Bilions)'}, inplace=True)
print(df_combined_daily_btc.head().to_markdown())
print(df_combined_daily_eth.head().to_markdown())


| Date                |    Open |    High |     Low |   Close |   Volume (Bilions) |   Fear & Greed Index |   Hashrate_EH |   GTrends_BTC |
|:--------------------|--------:|--------:|--------:|--------:|-------------------:|---------------------:|--------------:|--------------:|
| 2022-01-01 00:00:00 | 46311.7 | 47827.3 | 46288.5 | 47686.8 |            2.45827 |                   21 |       207.532 |            45 |
| 2022-01-02 00:00:00 | 47680.9 | 47881.4 | 46856.9 | 47345.2 |            2.79516 |                   29 |       190.64  |            61 |
| 2022-01-03 00:00:00 | 47343.5 | 47510.7 | 45836   | 46458.1 |            3.30716 |                   29 |       168.922 |            61 |
| 2022-01-04 00:00:00 | 46458.9 | 47406.5 | 45752.5 | 45897.6 |            4.24947 |                   23 |       177.368 |            61 |
| 2022-01-05 00:00:00 | 45899.4 | 46929   | 42798.2 | 43569   |            3.68511 |                   24 |       180.988 |            61 |
| Date              

In [13]:
print("For ETH:")
df_combined_daily_eth.info()
print("For BTC:")
df_combined_daily_btc.info()

For ETH:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1096 entries, 2022-01-01 to 2024-12-31
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Open                1096 non-null   float64
 1   High                1096 non-null   float64
 2   Low                 1096 non-null   float64
 3   Close               1096 non-null   float64
 4   Volume (Bilions)    1096 non-null   float64
 5   Fear & Greed Index  1096 non-null   float64
 6   GTrends_ETH         1094 non-null   float64
dtypes: float64(7)
memory usage: 68.5 KB
For BTC:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1096 entries, 2022-01-01 to 2024-12-31
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Open                1096 non-null   float64
 1   High                1096 non-null   float64
 2   Low                 1096 non-null   float64
 3   Cl

START OF LEARNING MODEL

In [ ]:
from pmdarima import auto_arima
import numpy as np


In [ ]:
#Training & Testing for BTC model

b_train = df_combined_daily_btc.loc['2022-01-01':'2023-']
b_test = df_combined_daily_btc.loc['2024-01-01':]